# Import data

In [68]:
import pandas as pd
import numpy as np
from pandas.api.types import infer_dtype

df = pd.read_csv('./csv_files/properties.csv', low_memory=False)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75876 entries, 0 to 75875
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   source              75876 non-null  int64  
 1   hyperlink           59908 non-null  object 
 2   locality            75874 non-null  object 
 3   postcode            68464 non-null  float64
 4   house_is            75876 non-null  object 
 5   property_subtype    75876 non-null  object 
 6   price               74405 non-null  object 
 7   sale                25810 non-null  object 
 8   rooms_number        74502 non-null  object 
 9   area                72349 non-null  object 
 10  kitchen_has         69763 non-null  object 
 11  furnished           69438 non-null  object 
 12  open_fire           70005 non-null  object 
 13  terrace             61138 non-null  object 
 14  terrace_area        64583 non-null  object 
 15  garden              67421 non-null  object 
 16  gard

# Check if there are columns with mixed data types ==> NO

In [69]:
# print data type of each column to check if there are
# any mixed ones, turns out that there are none
def is_mixed(col):
    return infer_dtype(col)

df.apply(is_mixed)

# ==> there are no columns with 'mixed' part of the inferred datatype

source                 integer
hyperlink               string
locality                string
postcode              floating
house_is                string
property_subtype        string
price                   string
sale                    string
rooms_number            string
area                    string
kitchen_has            boolean
furnished              boolean
open_fire              boolean
terrace                 string
terrace_area            string
garden                  string
garden_area             string
land_surface            string
land_plot_surface       string
facades_number          string
swimming_pool_has      boolean
building_state          string
dtype: object

# Remove leading and trailing spaces from column names

In [70]:
df.columns = [x.strip(' ') for x in df.columns.values]

# Find which kind of empties there are ==> there are only NaNs

In [71]:
# are there any empty strings? ==> no
print(np.where(df.applymap(lambda x: x == '')))

# are there any NaNs? ==> yes
np.where(pd.isnull(df))

(array([], dtype=int64), array([], dtype=int64))


(array([    0,     1,     2, ..., 75875, 75875, 75875]),
 array([ 7,  7,  7, ..., 19, 20, 21]))

# Display the percent of NaNs per column

In [72]:
# display the percent of NaNs per column
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'%_missing': percent_missing})
missing_value_df = missing_value_df.sort_values('%_missing', ascending = False)
missing_value_df

%_missing
sale               65.983974
building_state     41.092045
hyperlink          21.044863
garden_area        20.736465
terrace            19.423797
land_plot_surface  16.698297
land_surface       15.643945
terrace_area       14.883494
facades_number     12.495387
swimming_pool_has  11.986662
garden             11.143181
postcode            9.768570
furnished           8.484896
kitchen_has         8.056566
open_fire           7.737625
area                4.648374
price               1.938689
rooms_number        1.810849
locality            0.002636
property_subtype    0.000000
house_is            0.000000
source              0.000000

# Put 'unknown' in place of NaN for everything else than int64 and float64 columns
## Please run this after converting numerical columns like price and facades from string to integer.  
Even a value of NaN might help predict the price, so to avoid the correlation algorithm skipping it?, and because NaN is not allowed, we replace it.

In [73]:
# replace all NaNs in strings with 'unknown'
df_nanfilled = df.select_dtypes(exclude=['int64','float64']).replace(np.nan, 'unknown')
df.update(df_nanfilled)
print(df.info())

# replace all 'None'/'none' strings with uknown
df_nonefilled = df.select_dtypes(exclude=['int64','float64']).replace('none', 'unknown')
df.update(df_nonefilled)
df_nonefilled = df.select_dtypes(exclude=['int64','float64']).replace('None', 'unknown')
df.update(df_nonefilled)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75876 entries, 0 to 75875
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   source             75876 non-null  int64  
 1   hyperlink          75876 non-null  object 
 2   locality           75876 non-null  object 
 3   postcode           68464 non-null  float64
 4   house_is           75876 non-null  object 
 5   property_subtype   75876 non-null  object 
 6   price              75876 non-null  object 
 7   sale               75876 non-null  object 
 8   rooms_number       75876 non-null  object 
 9   area               75876 non-null  object 
 10  kitchen_has        75876 non-null  object 
 11  furnished          75876 non-null  object 
 12  open_fire          75876 non-null  object 
 13  terrace            75876 non-null  object 
 14  terrace_area       75876 non-null  object 
 15  garden             75876 non-null  object 
 16  garden_area        758

# Remove leading and trailing spaces of every element

In [74]:
# remove leading and trailing spaces and newline characters from values if they are a string
df = df.applymap(lambda x: x.strip() if type(x)==str else x)

# Drop postcode column, because postcode is more completely available in 'locality'
### first we fix 'locality' column to carry just postcode or 'unknown' (stripping sporadic address parts)

In [75]:
import re

df.drop('postcode', axis = 1, inplace = True)

# write a function that returns the cleaned postcode from elements
# containing the address
def clean_locality(locality): 
    # Search for the presence of a 4 digit number (starts with 1-9)
    if re.search('[1-9]\d{3}', locality):
        # get the number
        return re.findall("[1-9]\d{3}", locality)[0]
    else: 
        # if no postcode is inside insert 'unknown' 
        return 'unknown'
          
# Updated locality column
df['locality'] = df['locality'].apply(clean_locality)

# Remove duplicates
### should execute after fixing columns
### should execute after removing non-property detail or incomplete columns: source and hyperlink

In [76]:
# drop columns 
df.drop(['source', 'hyperlink', 'sale'], axis = 1, inplace = True)

# drop 100% duplicate rows
lenght_before = len(df)
df.drop_duplicates(ignore_index = True, inplace = True)
dropped = len(df) - lenght_before
print(f'Dropped: {dropped}')

Dropped: -23730


# Print unique values per column

In [77]:
uniques = pd.DataFrame()
for col in df:
    col_uniques = pd.DataFrame({f'{col}_value': df[f'{col}'].value_counts().index,
                                f'{col}_count': df[f'{col}'].value_counts().values})
    uniques = pd.concat([uniques, col_uniques], axis = 1)

uniques.head(50)


locality_value  locality_count house_is_value  house_is_count  \
0         unknown         23768.0          False         16305.0   
1            8300          1169.0           True         16013.0   
2            1180           942.0           TRUE          9463.0   
3            1000           731.0            Yes          4972.0   
4            1050           686.0             No          4734.0   
5            9000           570.0          FALSE           659.0   
6            8400           457.0            NaN             NaN   
7            4000           336.0            NaN             NaN   
8            1150           298.0            NaN             NaN   
9            1200           292.0            NaN             NaN   
10           1070           284.0            NaN             NaN   
11           8370           270.0            NaN             NaN   
12           1030           264.0            NaN             NaN   
13           4500           260.0            NaN             NaN   
14           8000           258.0            NaN             NaN   
15           9300           258.0            NaN             NaN   
16           1420           254.0            NaN             NaN   
17           8670           248.0            NaN             NaN   
18           2000           244.0            NaN             NaN   
19           1410           228.0            NaN             NaN   
20           9600           195.0            NaN             NaN   
21           1190           194.0            NaN             NaN   
22           9700           192.0            NaN             NaN   
23           8434           189.0            NaN             NaN   
24           1160           188.0            NaN             NaN   
25           1040           182.0            NaN             NaN   
26           1080           167.0            NaN             NaN   
27           6700           165.0            NaN             NaN   
28           2018           164.0            NaN             NaN   
29           4020           160.0            NaN             NaN   
30           8620           150.0            NaN             NaN   
31           1090           148.0            NaN             NaN   
32           1060           146.0            NaN             NaN   
33           8800           143.0            NaN             NaN   
34           3000           141.0            NaN             NaN   
35           8301           138.0            NaN             NaN   
36           8500           137.0            NaN             NaN   
37           1640           135.0            NaN             NaN   
38           1380           133.0            NaN             NaN   
39           7700           132.0            NaN             NaN   
40           1400           131.0            NaN             NaN   
41           2830           129.0            NaN             NaN   
42           8430           125.0            NaN             NaN   
43           9200           125.0            NaN             NaN   
44           8200           124.0            NaN             NaN   
45           9500           122.0            NaN             NaN   
46           1500           120.0            NaN             NaN   
47           1300           119.0            NaN             NaN   
48           7000           116.0            NaN             NaN   
49           3300           112.0            NaN             NaN   

   property_subtype_value  property_subtype_count price_value  price_count  \
0                   HOUSE                 15224.0     unknown        679.0   
1               APARTMENT                 10053.0      295000        468.0   
2                   house                  3711.0      199000        453.0   
3               apartment                  3646.0      299000        449.0   
4                   VILLA                  3258.0      275000        435.0   
5         APARTMENT_BLOCK                  1909.0      249000     